In [1]:
from dotenv import load_dotenv, find_dotenv
# read local .env file
_ = load_dotenv(find_dotenv())

In [2]:
from langchain_anthropic import ChatAnthropic
from langchain_core.pydantic_v1 import BaseModel, Field
from langchain_anthropic.output_parsers import ToolsOutputParser

In [3]:
opus_model_id = 'claude-3-opus-20240229'
sonnet_model_id = 'claude-3-sonnet-20240229'
haiku_model_id = 'claude-3-haiku-20240307'

In [4]:
# Data model
class PythonREPL(BaseModel):
    """A Python REPL that can be used to executed Python code"""
    code: str = Field(description="Code block to be executed in a Python REPL")

In [5]:
llm = ChatAnthropic(model=opus_model_id) #,default_headers={"anthropic-beta": "tools-2024-04-04"}
structured_llm = llm.with_structured_output(PythonREPL, include_raw=True)

llm_with_tools = llm.bind_tools([PythonREPL])

/Users/rosclay/Documents/VSCode/langgraph-multi-agent/venv/lib/python3.10/site-packages/langchain_core/_api/beta_decorator.py:87: LangChainBetaWarning: The function `with_structured_output` is in beta. It is actively being worked on, so the API may change.
  warn_beta(
/Users/rosclay/Documents/VSCode/langgraph-multi-agent/venv/lib/python3.10/site-packages/langchain_core/_api/beta_decorator.py:87: LangChainBetaWarning: The function `bind_tools` is in beta. It is actively being worked on, so the API may change.
  warn_beta(


In [8]:
llm_with_both = llm_with_tools.with_structured_output(PythonREPL, include_raw=True)

AttributeError: 'RunnableBinding' object has no attribute 'with_structured_output'

In [6]:
structured_llm.dict()

{'name': None,
 'first': {'name': None,
  'steps': {'raw': {'name': None,
    'bound': {'_type': 'anthropic-chat'},
    'kwargs': {'tools': [{'name': 'PythonREPL',
       'description': 'A Python REPL that can be used to executed Python code',
       'input_schema': {'type': 'object',
        'properties': {'code': {'description': 'Code block to be executed in a Python REPL',
          'type': 'string'}},
        'required': ['code']}}]},
    'config': {},
    'config_factories': [],
    'custom_input_type': None,
    'custom_output_type': None}}},
 'middle': [],
 'last': {'name': None,
  'runnable': {'name': None,
   'mapper': {'name': None,
    'steps': {'parsed': {'name': None,
      'first': RunnableLambda(itemgetter('raw')),
      'middle': [],
      'last': {'name': None,
       'first_tool_only': True,
       'args_only': False,
       'pydantic_schemas': [__main__.PythonREPL]}},
     'parsing_error': RunnableLambda(...)}}},
  'fallbacks': [{'name': None,
    'mapper': {'name': 

In [7]:
llm_with_tools.dict()

{'name': None,
 'bound': {'_type': 'anthropic-chat'},
 'kwargs': {'tools': [{'name': 'PythonREPL',
    'description': 'A Python REPL that can be used to executed Python code',
    'input_schema': {'type': 'object',
     'properties': {'code': {'description': 'Code block to be executed in a Python REPL',
       'type': 'string'}},
     'required': ['code']}}]},
 'config': {},
 'config_factories': [],
 'custom_input_type': None,
 'custom_output_type': None}

In [ ]:
result = llm_with_tools.invoke('write a python function to square a number')

In [ ]:
result

In [ ]:
result.content

In [ ]:
chain = llm_with_tools | ToolsOutputParser()

In [ ]:
response = chain.invoke('write a python function to square a number')

In [ ]:
response

In [ ]:
response = llm_with_tools.invoke('hi how are you')

In [ ]:
print(response.content)

In [ ]:
plan = '''1. Get all pitch data from 2020-08-01 to 2020-08-07:
all_pitches = statcast('2020-08-01', '2020-08-07')

2. Filter for just curveballs:
all_curves = all_pitches[all_pitches['pitch_type'] == 'CU']

3. Create a feature vector for each pitcher's curveballs:
pitcher_curves = all_curves.groupby('pitcher')
pitcher_features = pitcher_curves[['release_speed', 'release_spin', 'pfx_x', 'pfx_z']].mean().reset_index()

4. Get Max Scherzer's player ID:
from pybaseball import playerid_lookup
scherzer_id = playerid_lookup('scherzer', 'max')['key_mlbam'][0]

5. Get Scherzer's feature vector:
scherzer_features = pitcher_features[pitcher_features['pitcher'] == scherzer_id]

6. Import scikit-learn and create a knn model:
from sklearn.neighbors import NearestNeighbors
knn = NearestNeighbors(n_neighbors=4)
knn.fit(pitcher_features[['release_speed', 'release_spin', 'pfx_x', 'pfx_z']])

7. Find the 3 pitchers closest to Scherzer:
distances, indices = knn.kneighbors(scherzer_features[['release_speed', 'release_spin', 'pfx_x', 'pfx_z']])
closest_indices = indices[0][1:4]
similar_pitchers = pitcher_features.iloc[closest_indices]
print(similar_pitchers[['pitcher']])'''

In [ ]:
result = chain.invoke(f'Execute the next step of this plan:\n\n{plan}')

In [ ]:
result

In [ ]:
print(result[0]['args']['code'])

In [ ]:
from dotenv import load_dotenv, find_dotenv

#from langchain_community.chat_models import BedrockChat
from langchain_anthropic import ChatAnthropic
from langchain_core.messages import HumanMessage
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder

# read local .env file
_ = load_dotenv(find_dotenv())

# define language model
model_id = 'anthropic.claude-3-sonnet-20240229-v1:0'
#model_id = 'anthropic.claude-3-haiku-20240307-v1:0'
#llm = BedrockChat(model_id=model_id, model_kwargs={'temperature': 0})

opus_model_id = 'claude-3-opus-20240229'
sonnet_model_id = 'claude-3-sonnet-20240229'
haiku_model_id = 'claude-3-haiku-20240307'

llm = ChatAnthropic(model=opus_model_id, temperature=0)

In [ ]:
llm.invoke('what day comes after tuesday?')